# EDA

In [64]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ast

# Load datasets

In [65]:
path = "literate-dollop/movies/"
movies = pd.read_csv(f"{path}movies_metadata.csv", low_memory=False)
credits = pd.read_csv(f"{path}credits.csv")
keywords = pd.read_csv(f"{path}keywords.csv")
links = pd.read_csv(f"{path}links.csv")
ratings = pd.read_csv(f"{path}ratings_small.csv")

# Parse json

In [66]:
def parse_json(x):
    if pd.isna(x): return []
    try:
        return ast.literal_eval(x)
    except: 
        return []
    
for col in ['genres','production_companies','production_countries','spoken_languages','belongs_to_collection']:
    movies[col] = movies[col].apply(parse_json)

for col in ['cast', 'crew']:
    credits[col] = credits[col].apply(parse_json)

# String values to int, NaN to 0

In [67]:
for col in ['budget','revenue','runtime','vote_average','vote_count']:
    movies[col] = pd.to_numeric(movies[col], errors='coerce').fillna(0)

# Convert release_date to datetime
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')

movies["id"] = pd.to_numeric(movies["id"], errors="coerce")
links["movieId"] = pd.to_numeric(links["movieId"], errors="coerce")
links["tmdbId"] = pd.to_numeric(links["tmdbId"], errors="coerce")
links["imdbId"] = pd.to_numeric(links["imdbId"], errors="coerce")


## Check if there are movies in links that are not in movies

In [68]:
movies_ids = pd.Series(movies['id'].dropna().astype('Int64').unique())
links_tmdb = pd.Series(links['tmdbId'].dropna().astype('Int64').unique())

movies_set = set(movies_ids)
links_set = set(links_tmdb)

missing_in_links = sorted(movies_set - links_set)
extra_in_links = sorted(links_set - movies_set)

print(f"Total unique movie IDs in movies: {len(movies_set)}")
print(f"Total unique tmdbIds in links: {len(links_set)}")
print(f"Number of movie IDs missing in links: {len(missing_in_links)}")
print(f"Number of tmdbIds in links not present in movies: {len(extra_in_links)}")

if missing_in_links:
    print("\nSample missing movie IDs (up to 20):", missing_in_links[:20])
    display(movies[movies['id'].isin(missing_in_links)][['id', 'title', 'release_date']])

if extra_in_links:
    print("\nSample extra tmdbIds in links not in movies (up to 20):", extra_in_links[:20])
    display(links[links['tmdbId'].isin(extra_in_links)].head(20))

Total unique movie IDs in movies: 45433
Total unique tmdbIds in links: 45594
Number of movie IDs missing in links: 0
Number of tmdbIds in links not present in movies: 161

Sample extra tmdbIds in links not in movies (up to 20): [np.int64(2413), np.int64(2518), np.int64(2670), np.int64(10700), np.int64(12188), np.int64(12773), np.int64(13057), np.int64(13716), np.int64(15533), np.int64(15594), np.int64(15738), np.int64(17882), np.int64(19341), np.int64(20881), np.int64(20892), np.int64(21680), np.int64(24219), np.int64(24549), np.int64(25093), np.int64(29133)]


,movieId,imdbId,tmdbId
4114,4207,100232,12773.0
4474,4568,96913,17882.0
4974,5069,270933,68149.0
5113,5209,81809,24549.0
7416,7669,112130,164721.0
8812,26379,75520,2413.0
8855,26452,83387,82205.0
8973,26649,96639,13057.0
8994,26693,99864,2670.0
9187,27049,135659,15738.0


In [69]:
movies[movies["id"]==64699.0]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count


Movie exists in links but not in movies

## Check if person ID is unique

In [70]:
# Flatten both lists into one dataframe of (id, name, job_type)
people_records = []

for _, row in credits.iterrows():
    for person in row["cast"]:
        if person["id"] is not None:
            people_records.append((person["id"], person["name"], "cast"))
    for person in row["crew"]:
        if person["id"] is not None:
            people_records.append((person["id"], person["name"], "crew"))

people_df = pd.DataFrame(people_records, columns=["id", "name", "type"])

# Group by ID and count distinct names
duplicates = (
    people_df.groupby("id")["name"]
    .nunique()
    .reset_index(name="unique_names")
)

# IDs mapping to multiple names (inconsistent)
inconsistent = duplicates[duplicates["unique_names"] > 1]

print(f"Inconsistent person IDs: {len(inconsistent)}")
if not inconsistent.empty:
    print("\nExamples of mismatched IDs:")
    for pid in inconsistent["id"].head(10):
        names = people_df.loc[people_df["id"] == pid, "name"].unique()


        print(f"ID {pid}: {names}")
else:
    print("All person IDs consistently map to one name.")

Inconsistent person IDs: 15

Examples of mismatched IDs:
ID 9779: ['Моррис Честнат' 'Morris Chestnut']
ID 23764: ['Эрика Элениак' 'Erika Eleniak']
ID 58646: ['Дэмиан Чапа' 'Damian Chapa']
ID 63574: ['Ka-Fai Cheung' 'Cheung Ka-Fai']
ID 67212: ['Том Ву' 'Tom Wu']
ID 72932: ['松田龙平' 'Ryuhei Matsuda']
ID 74947: ['张雨绮' 'Kitty Zhang Yuqi']
ID 78456: ['Кевин «Кимбо Слайс» Фергюсон' 'Kimbo Slice']
ID 78809: ['Рашад Эванс' 'Rashad Evans']
ID 87637: ['长泽雅美' 'Masami Nagasawa']


Resultatet her viser at det kun er alternative måter å skrive navnene på

# Clean data

## Drop movies corrputed movies with no ID

In [71]:
movies["id"] = pd.to_numeric(movies["id"], errors="coerce")
links["movieId"] = pd.to_numeric(links["movieId"], errors="coerce")
links["tmdbId"] = pd.to_numeric(links["tmdbId"], errors="coerce")
links["imdbId"] = pd.to_numeric(links["imdbId"], errors="coerce")

missing_tmdb = movies["id"].isna().sum()

missing_movie = links["movieId"].isna().sum()
missing_imdb = links["imdbId"].isna().sum()
missing_tmdblinks = links["tmdbId"].isna().sum()

print(f"Missing TMDB IDs in movies_metadata: {missing_tmdb}")
print(f"Missing MovieLens IDs in links: {missing_movie}")
print(f"Missing IMDb IDs in links: {missing_imdb}")
print(f"Missing TMDB IDs in links: {missing_tmdblinks}")

print("\nMovies with missing TMDB ID:")
print(movies[movies["id"].isna()][["title", "release_date"]])

print("\nLinks with missing IMDb ID:")
print(links[links["imdbId"].isna()])

movies[movies["id"].isna()]

Missing TMDB IDs in movies_metadata: 3
Missing MovieLens IDs in links: 0
Missing IMDb IDs in links: 0
Missing TMDB IDs in links: 219

Movies with missing TMDB ID:
      title release_date
19730   NaN          NaT
29503   NaN          NaT
35587   NaN          NaT

Links with missing IMDb ID:
Empty DataFrame
Columns: [movieId, imdbId, tmdbId]
Index: []


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,0.0,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",NaN,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,NaT,0.0,0.0,[],NaN,NaN,NaN,NaN,0.0,0.0
29503,Rune Balot goes to a casino connected to the ...,1.931659,0.0,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",NaN,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,NaT,0.0,0.0,[],NaN,NaN,NaN,NaN,0.0,0.0
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,0.0,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",NaN,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,NaT,0.0,0.0,[],NaN,NaN,NaN,NaN,0.0,0.0


In [72]:
movies = movies.dropna(subset=["id"])

## Check for duplicate movies

In [73]:
duplicates = movies[movies.duplicated(subset="id", keep=False)]
print(f"Duplicate rows: {len(duplicates)}")
display(duplicates)

# If you want only the duplicate ids
dup_ids = movies[movies.duplicated(subset="id", keep=False)]["id"].unique()
print("Duplicate IDs:", dup_ids)

Duplicate rows: 59


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
676,False,[],0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,105045.0,tt0111613,de,Das Versprechen,"East-Berlin, 1961, shortly after the erection ...",...,1995-02-16,0.0,115.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Released,"A love, a hope, a wall.",The Promise,False,5.0,1.0
838,False,[],0.0,"[{'id': 18, 'name': 'Drama'}]",NaN,132641.0,tt0046468,ja,Tsuma,"Ten years into a marriage, the wife is disappo...",...,1953-04-29,0.0,89.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Wife,False,0.0,0.0
949,False,[],4.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,22649.0,tt0022879,en,A Farewell to Arms,British nurse Catherine Barkley (Helen Hayes) ...,...,1932-12-08,25.0,89.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Every woman who has loved will understand,A Farewell to Arms,False,6.2,29.0
1465,False,[],0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,105045.0,tt0111613,de,Das Versprechen,"East-Berlin, 1961, shortly after the erection ...",...,1995-02-16,0.0,115.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Released,"A love, a hope, a wall.",The Promise,False,5.0,1.0
2564,False,[],0.0,"[{'id': 99, 'name': 'Documentary'}]",NaN,84198.0,tt1736049,en,A Place at the Table,"Using personal stories, this powerful document...",...,2012-03-22,0.0,84.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,One Nation. Underfed.,A Place at the Table,False,6.9,7.0
4114,False,"{'id': 34055, 'name': 'Pokémon Collection', 'p...",16000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://movies.warnerbros.com/pk3/,10991.0,tt0235679,ja,Pokémon 3: The Movie,When Molly Hale's sadness of her father's disa...,...,2000-07-08,68411275.0,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Pokémon: Spell of the Unknown,Pokémon: Spell of the Unknown,False,6.0,143.0
4356,False,[],3512454.0,"[{'id': 18, 'name': 'Drama'}]",NaN,110428.0,tt2018086,fr,Camille Claudel 1915,"Winter, 1915. Confined by her family to an asy...",...,2013-03-13,115860.0,95.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,NaN,Camille Claudel 1915,False,7.0,20.0
5130,False,[],26000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 10751...",NaN,15028.0,tt0157472,en,Clockstoppers,"Until now, Zak Gibbs' greatest challenge has b...",...,2002-03-17,38793283.0,94.0,"[{'iso_639_1': 'cs', 'name': 'Český'}, {'iso_6...",Released,"The adventure of a lifetime, in a few mere sec...",Clockstoppers,False,4.9,89.0
5535,False,"{'id': 34055, 'name': 'Pokémon Collection', 'p...",0.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://www.pokemon.com/us/movies/movie-pokemon...,12600.0,tt0287635,ja,劇場版ポケットモンスター セレビィ 時を越えた遭遇（であい）,"All your favorite Pokémon characters are back,...",...,2001-07-06,28023563.0,75.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Pokémon 4Ever: Celebi - Voice of the Forest,False,5.7,82.0
5710,False,[],0.0,"[{'id': 18, 'name': 'Drama'}]",NaN,109962.0,tt0082992,en,Rich and Famous,Two literary women compete for 20 years: one w...,...,1981-09-23,0.0,115.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"From the very beginning, they knew they'd be f...",Rich and Famous,False,4.9,7.0


Duplicate IDs: [105045. 132641.  22649.  84198.  10991. 110428.  15028.  12600. 109962.
   4912.   5511.  23305.  69234.  14788.  77221.  13209. 159849. 141971.
 168538.  97995.  18440.  11115.  42495.  99080.  25541. 119916. 152795.
 265189. 298721.]


# Drop duplicates

In [74]:
# Remove duplicate movie IDs, keep first occurrence
movies = movies.drop_duplicates(subset='id', keep='first')


# Prepare for db